In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
#pd.set_option('display.max_columns', None)

Dans le cadre du data challenge QRT de cette année, nous vous proposons un challenge de prédiction de résultat de match. Vous recevrez des données historiques réelles au niveau des équipes et des joueurs, et vous devrez prédire quelle équipe gagne ou s'il y a un match nul.

Le score cible est l’accuracy de la prédiction pour les trois classes [HOME_WINS, DRAW, AWAY_WINS], Il existe donc pour un match trois outputs possibles, [1,0,0]. [0,1,0] et [0,0,1].

In [18]:
lien = '/Users/augustincablant/Desktop/QRT/train'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename, index_col=0))

player_away, player_home, team_away, team_home, y_train = liste_df[0], liste_df[1], liste_df[2], liste_df[4], liste_df[3]

for col in player_away.columns:
    if col not in player_home.columns:
        print(col)

for col in team_away.columns:
    if col not in team_home.columns:
        print(col)

#Ces deux datasets ont les mêmes colonnes. On créer donc la colonne home qui vaut True ou False et on agrège les deux datasets !

player_away['HOME'], player_home['HOME'] = False, True
df_player = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team = pd.concat([team_home, team_away])


##test 

lien = '/Users/augustincablant/Desktop/QRT/test'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename, index_col=0))

team_away, player_home, team_home, player_away = liste_df[0], liste_df[1], liste_df[2], liste_df[3]

player_away['HOME'], player_home['HOME'] = False, True
df_player_test = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team_test = pd.concat([team_home, team_away])

df_team, df_team_test = df_team.loc[:, ~df_team.columns.duplicated()], df_team_test.loc[:, ~df_team_test.columns.duplicated()]

/Users/augustincablant/Desktop/QRT/train/train_away_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/Y_train.csv
/Users/augustincablant/Desktop/QRT/train/train_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_player_statistics_df.csv


In [19]:
df_team['HOME'].value_counts()

True     12303
False    12303
Name: HOME, dtype: int64

## Idée numéro 1 

On construit un dataset sur le dataset df_team. 
Pour évalue le niveau de chaque joueur de chaque team grâce à une métrique que l'on construit, puis on fait une moyenne sur l'ensemble des joueurs de l'équipe.

Attention, dans le dataset test nous n'avons pas accès au nom des équipes. 

In [20]:
# Le but de cette cellule de code est de : 
# Construire une métrique basée sur les joueurs du match (attention bien prendre en compte away et home)
# Ajouter cette valeur dans le dataset df_team
# Faire de même pour le dataset test

#TRAIN
IDS = df_team.index.unique()
df_team_metric = df_team.copy()
df_team_metric['PLAYER_METRIC'] = np.nan

for id in tqdm(IDS):
    sub_true, sub_false = df_player[(df_player.index ==id) & (df_player['HOME']==True)], df_player[(df_player.index ==id) & (df_player['HOME']==False)]
    sub_numeric_true,  sub_numeric_false = sub_true.drop(columns=['LEAGUE', 'TEAM_NAME', 'POSITION', 'PLAYER_NAME', 'HOME']), sub_false.drop(columns=['LEAGUE', 'TEAM_NAME', 'POSITION', 'PLAYER_NAME', 'HOME'])
    mean_true, mean_false = sub_numeric_true.mean(axis=1, skipna=True).mean(), sub_numeric_false.mean(axis=1, skipna=True).mean()
    df_team_metric.loc[(df_team_metric.index == id) & (df_team_metric['HOME'] == True), 'PLAYER_METRIC'] = mean_true
    df_team_metric.loc[(df_team_metric.index  == id) & (df_team_metric['HOME'] == False), 'PLAYER_METRIC'] = mean_false


100%|██████████| 12303/12303 [00:35<00:00, 350.25it/s]


In [21]:
#TEST (très long)
IDS_test = df_team_test.index.unique()
df_team_metric_test = df_team_test.copy()
df_team_metric_test['PLAYER_METRIC'] = np.nan

for id in tqdm(IDS_test):
    sub_true, sub_false = df_player_test[(df_player_test.index ==id) & (df_player_test['HOME']==True)], df_player_test[(df_player_test.index ==id) & (df_player_test['HOME']==False)]
    sub_numeric_true,  sub_numeric_false = sub_true.drop(columns=['POSITION', 'HOME']), sub_false.drop(columns=['POSITION', 'HOME'])
    mean_true, mean_false = sub_numeric_true.mean(axis=1, skipna=True).mean(), sub_numeric_false.mean(axis=1, skipna=True).mean()
    df_team_metric_test.loc[(df_team_metric_test.index  == id) & (df_team_metric_test['HOME'] == True), 'PLAYER_METRIC'] = mean_true
    df_team_metric_test.loc[(df_team_metric_test.index  == id) & (df_team_metric_test['HOME'] == False), 'PLAYER_METRIC'] = mean_false

100%|██████████| 25368/25368 [01:24<00:00, 301.50it/s]


In [22]:
df_train = pd.merge(df_team_metric, y_train, left_index=True, right_index=True, how='left').fillna(0)
df_train['TARGET'] = df_train['HOME_WINS'] - df_train['AWAY_WINS'] 
df_train['HOME'] = df_train['HOME'].astype(float)
X, y = df_train.drop(columns=['LEAGUE', 'TEAM_NAME', 'HOME_WINS', 'DRAW', 'AWAY_WINS', 'TARGET']).copy(), df_train['TARGET']

In [119]:
rf = RandomForestClassifier()
X_train, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [200],
    'max_depth': [40, 50, 60],
    'min_samples_split': [20, 30, 40],
    'min_samples_leaf': [10, 15, 20],
    'bootstrap': [False]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_tr)
print("Meilleurs paramètres trouvés :", grid_search.best_params_)
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle avec les meilleurs paramètres :", accuracy)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=30, n_estimators=200; total time=  16.2s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=30, n_estimators=200; total time=  16.5s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=30, n_estimators=200; total time=  16.8s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=20, n_estimators=200; total time=  16.9s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=20, n_estimators=200; total time=  16.9s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=20, n_estimators=200; total time=  17.0s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=20, n_estimators=200; total time=  17.3s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=10, min_samples_split=20, n_estimators=20

In [23]:
X_test = df_team_metric_test.copy().fillna(0)
model = RandomForestClassifier(bootstrap= False, 
                               max_depth= 40, 
                               min_samples_leaf = 15, 
                               min_samples_split= 30, 
                               n_estimators= 200)
model.fit(X,y)
predictions = model.predict(X_test)

def transform_target(row):
    if row==1:
        return [1,0,0]
    elif row==0:
        return [0,1,0]
    elif row==-1:
        return [0,0,1]
    
    
pred = [transform_target(value) for value in predictions]
predictions = pd.DataFrame(pred)
predictions.columns = ['HOME_WINS', 'DRAW', 'AWAY_WINS']
predictions.index = df_team_test.index
submission = predictions.reset_index()

submission.to_csv('/Users/augustincablant/Desktop/QRT/submission1.csv', index=False)


Idée : on voit déjà que prédire toujours la victoire marche bien. 
Je pense qu'il faudrait essayer de se focus sur les matchs nuls, parvenir à trouver leur singularité ? Peut-être le fait que les stats des deux équipes soient très proches ? 

In [189]:
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
X_train, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

for value in [10, 20, 30, 40, 50, 60, 70, 80]:
    model_test = RandomForestClassifier(bootstrap= False, 
                               max_depth= 40, 
                               min_samples_leaf = 15, 
                               min_samples_split= 30, 
                               n_estimators= 200)
    top_features = feature_importances.head(value)['feature']
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]
    model_test.fit(X_train_selected, y_tr)
    y_pred = model_test.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for value = {value} : ", accuracy)


Accuracy for value = 10 :  0.45306785859406745
Accuracy for value = 20 :  0.45388053636733033
Accuracy for value = 30 :  0.46038195855343356
Accuracy for value = 40 :  0.4656643640796424
Accuracy for value = 50 :  0.4697277529459569
Accuracy for value = 60 :  0.4620073140999594
Accuracy for value = 70 :  0.4648516863063795
Accuracy for value = 80 :  0.4656643640796424


In [192]:
model = RandomForestClassifier(bootstrap= False, 
                               max_depth= 40, 
                               min_samples_leaf = 15, 
                               min_samples_split= 30, 
                               n_estimators= 200)
X_test = df_team_metric_test.copy().fillna(0)
top_features = feature_importances.head(50)['feature']
X_train_selected = X[top_features]
X_test_selected = X_test[top_features]
model.fit(X_train_selected, y)
y_pred = model.predict(X_test_selected)
pred = [transform_target(value) for value in y_pred]
predictions = pd.DataFrame(pred)
predictions.columns = ['HOME_WINS', 'DRAW', 'AWAY_WINS']
predictions.index = df_team_test.index
submission = predictions.reset_index()

submission.to_csv('/Users/augustincablant/Desktop/QRT/submission2.csv', index=False)

## Idée numéro 2 

Procéder sur les matchs et donc faire une seule prédiction pour un ID. On compare les stats des deux équipes.

In [9]:
df_team

,LEAGUE,TEAM_NAME,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,...,TEAM_REDCARDS_5_last_match_std,TEAM_OFFSIDES_5_last_match_std,TEAM_ATTACKS_5_last_match_std,TEAM_PENALTIES_5_last_match_std,TEAM_SUBSTITUTIONS_5_last_match_std,TEAM_BALL_SAFE_5_last_match_std,TEAM_DANGEROUS_ATTACKS_5_last_match_std,TEAM_INJURIES_5_last_match_std,TEAM_GOALS_5_last_match_std,HOME
ID,,,,,,,,,,,,,,,,,,,,,
0,Ligue 1,Olympique Marseille,4.0,1.0,4.0,1.0,5.0,4.0,3.0,6.0,...,5.0,4.0,0.0,6.0,8.0,4.0,3.0,2.0,3.0,False
1,Ligue 2,Red Star,4.0,3.0,4.0,3.0,8.0,8.0,7.0,8.0,...,0.0,3.0,1.0,8.0,4.0,10.0,0.0,5.0,3.0,False
2,Serie A,Bologna,4.0,3.0,4.0,2.0,6.0,4.0,4.0,1.0,...,10.0,4.0,4.0,0.0,8.0,3.0,0.0,9.0,6.0,False
3,League One,Bolton Wanderers,0.0,0.0,0.0,0.0,1.0,3.0,3.0,8.0,...,0.0,1.0,2.0,0.0,5.0,6.0,3.0,NaN,2.0,False
4,Premier League,Leicester City,5.0,4.0,5.0,6.0,5.0,6.0,6.0,4.0,...,0.0,4.0,4.0,9.0,4.0,1.0,4.0,6.0,5.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,League One,Accrington Stanley,6.0,6.0,5.0,7.0,7.0,4.0,3.0,4.0,...,5.0,0.0,1.0,6.0,10.0,1.0,1.0,NaN,2.0,False
12299,Liga Portugal,Marítimo,3.0,2.0,6.0,1.0,3.0,4.0,4.0,7.0,...,5.0,2.0,2.0,0.0,0.0,1.0,6.0,9.0,1.0,False
12300,Bundesliga,RB Leipzig,7.0,7.0,6.0,7.0,3.0,3.0,2.0,4.0,...,0.0,3.0,2.0,0.0,0.0,6.0,3.0,4.0,4.0,False


In [8]:
team_home

,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,TEAM_CORNERS_season_sum,TEAM_FOULS_season_sum,...,TEAM_REDCARDS_5_last_match_std,TEAM_OFFSIDES_5_last_match_std,TEAM_ATTACKS_5_last_match_std,TEAM_PENALTIES_5_last_match_std,TEAM_SUBSTITUTIONS_5_last_match_std,TEAM_BALL_SAFE_5_last_match_std,TEAM_DANGEROUS_ATTACKS_5_last_match_std,TEAM_INJURIES_5_last_match_std,TEAM_GOALS_5_last_match_std,HOME
ID,,,,,,,,,,,,,,,,,,,,,
12303,3.0,6.0,5.0,1.0,3.0,9.0,8.0,10.0,6.0,7.0,...,9.0,0.0,10.0,0.0,4.0,4.0,6.0,2.0,3.0,True
12304,3.0,2.0,4.0,2.0,6.0,1.0,1.0,2.0,4.0,10.0,...,10.0,3.0,1.0,8.0,3.0,8.0,1.0,0.0,2.0,True
12305,7.0,10.0,8.0,8.0,1.0,4.0,5.0,0.0,10.0,4.0,...,8.0,2.0,9.0,0.0,6.0,0.0,9.0,NaN,0.0,True
12306,5.0,7.0,5.0,9.0,5.0,7.0,6.0,3.0,5.0,8.0,...,5.0,2.0,3.0,8.0,0.0,2.0,2.0,2.0,3.0,True
12307,5.0,4.0,6.0,3.0,5.0,0.0,0.0,2.0,2.0,9.0,...,0.0,0.0,7.0,5.0,0.0,1.0,6.0,7.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37666,5.0,5.0,5.0,5.0,7.0,6.0,6.0,4.0,6.0,8.0,...,6.0,2.0,1.0,5.0,6.0,6.0,5.0,2.0,0.0,True
37667,5.0,5.0,2.0,8.0,3.0,7.0,7.0,2.0,2.0,2.0,...,0.0,2.0,8.0,0.0,0.0,4.0,8.0,10.0,10.0,True
37668,10.0,10.0,10.0,8.0,8.0,5.0,5.0,2.0,6.0,0.0,...,0.0,8.0,4.0,0.0,7.0,NaN,2.0,NaN,3.0,True
